In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from scipy.stats import wilcoxon
from matplotlib.lines import Line2D

In [2]:
df = pd.read_csv('simulation_results_with_distances.csv', header=0)

In [3]:
df

,simulation,trial,start_time,end_time,duration,duration_no_distance,type,sequence,duration_seconds,diff
0,1,0,02:20:17 PM,02:26:15 PM,00:05:58,358,opm,"bowl, breakfast-cereal, cup, spoon, milk",362.743217,4.743217
1,1,1,02:26:17 PM,02:31:24 PM,00:05:07,307,opm,"bowl, breakfast-cereal, cup, spoon, milk",311.743217,4.743217
2,1,2,02:31:26 PM,02:36:32 PM,00:05:06,306,opm,"bowl, breakfast-cereal, cup, spoon, milk",310.743217,4.743217
3,1,3,02:36:36 PM,02:41:44 PM,00:05:08,308,opm,"bowl, breakfast-cereal, cup, spoon, milk",312.743217,4.743217
4,1,4,02:41:47 PM,02:46:15 PM,00:04:28,268,opm,"bowl, breakfast-cereal, cup, spoon, milk",272.743217,4.743217
...,...,...,...,...,...,...,...,...,...,...
95,5,5,01:38:15 PM,01:46:01 PM,00:07:46,466,baseline,"breakfast-cereal, cup, bowl, spoon, milk",471.309578,5.309578
96,5,6,01:46:03 PM,01:54:11 PM,00:08:08,488,baseline,"breakfast-cereal, cup, bowl, spoon, milk",493.309578,5.309578
97,5,7,01:54:13 PM,02:01:31 PM,00:07:18,438,baseline,"breakfast-cereal, cup, bowl, spoon, milk",443.309578,5.309578
98,5,8,02:01:33 PM,02:09:04 PM,00:07:31,451,baseline,"breakfast-cereal, cup, bowl, spoon, milk",456.309578,5.309578


In [4]:
custom_cmap2 = ["#46327e", "#277f8e", "#4ac16d"]

In [5]:
df[['duration_seconds', 'type']].groupby(['type']).mean()

,duration_seconds
type,
baseline,385.034155
opm,379.692473


In [6]:
duration_opm = df.loc[df['type'] == 'opm']['duration_seconds'].mean()

In [7]:
duration_baseline = df.loc[df['type'] == 'baseline']['duration_seconds'].mean()

In [8]:
durations = pd.Series([duration_baseline, duration_opm])

In [9]:
durations.pct_change() * 100

0         NaN
1   -1.387327
dtype: float64

In [10]:
opm = df['duration_seconds'].loc[df['type'] == 'opm'].tolist()
baseline = df['duration_seconds'].loc[df['type'] == 'baseline'].tolist()
len(opm), len(baseline)

(50, 50)

### stat analysis

In [11]:
stat, p = wilcoxon(opm, baseline)
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 533.000, p = 0.31832


### plot results

In [12]:
results = pd.DataFrame()

In [13]:
custom_cmap2 = ["#46327e", "#277f8e", "#4ac16d"]
custom_lines2 = [Line2D([0], [0], color=custom_cmap2[1], lw=8, alpha=0.5),
                Line2D([0], [0], color=custom_cmap2[2], lw=8, alpha=0.5)]

In [14]:
results['opm'] = opm
results['baseline'] = baseline
labels = [col for col in results.columns]

In [15]:
%matplotlib qt

ax = sns.violinplot(data=results,
                    palette='viridis',
                    cut=0,
                    inner='quartiles')

plt.setp(ax.collections, alpha=0.7)

ax.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize=24)

plt.ylabel('duration in seconds', fontsize=24)
#plt.tight_layout(pad=0.9)
plt.show()

In [16]:
simulations_opm = df['simulation'].loc[df['type'] == 'opm'].tolist()
simulations_baseline = df['simulation'].loc[df['type'] == 'baseline'].tolist()

In [17]:
simulations_opm == simulations_baseline

True

In [18]:
results['simulation'] = simulations_baseline

In [19]:
results.groupby('simulation').mean()

,opm,baseline
simulation,,
1,329.343217,371.561170
2,345.562443,345.348254
3,348.475510,335.270628
4,439.020027,424.681144
5,436.061170,448.309578


In [30]:
labels = [col for col in results.columns]

%matplotlib qt

ax2 = sns.violinplot(data=results.groupby('simulation').mean(),
                    palette=custom_cmap2[1:],
                    cut=1,
                    inner='quartiles',
                    saturation=.8)

plt.setp(ax2.collections, alpha=0.5)

#ax2.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize=20)
#ax2.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=20)
ax2.tick_params(axis='both', which='major', labelsize=20)

plt.ylabel('duration in seconds', fontsize=24)
plt.xlabel('simulation type', fontsize=24)

ax2.legend(custom_lines2, ['OPM', 'baseline'], loc='upper left', fontsize=24)

plt.show()

In [21]:
%matplotlib qt

ax2 = plt.figure(figsize=(12,8))

ax2 = sns.violinplot(x='simulation',
                     y='duration_seconds',
                     hue='type',
                     data=df,
                     palette=custom_cmap2[1:],
                     cut=0,
                     inner='quartiles',
                     saturation=.8)

#ax2.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize=18)

plt.ylabel('duration in seconds', fontsize=26)
plt.xlabel('simulation', fontsize=24)

plt.setp(ax2.collections, alpha=0.5)
ax2.legend(custom_lines2, ['OPM', 'baseline'], loc='upper left', fontsize=24)
ax2.tick_params(axis='both', which='major', labelsize=20)

#ax2 = plt.gca()
#leg = ax2.get_legend()
#leg.legendHandles[0].set_color(custom_cmap2[1])
#leg.legendHandles[1].set_color(custom_cmap2[2])

#legend = plt.legend()
#for lh in legend.legendHandles:
#    lh.set_alpha(0.5)

plt.tight_layout()
plt.margins(0.01)
plt.show()